### Group ID: Group 19
### Group Members Name with Student ID:
| No. | Name                        | BITS ID     | Contribution  |
|-----|-----------------------------|-----------------|--------|
| 1   | SHIDHAYE GAYATRI PRASHANT  | 2023AC05156     | 100%   |
| 2   | Smitha K                   | 2023AC05264     | 100%   |
| 3   | Revathi P                  | 2023AD05044     | 100%   |
| 4   | Yathish K T                | 2023AC05240     | 100%   |

Remarks: ##Add here

# Background

In the world of online streaming, user satisfaction and engagement are critical metrics for the success of a movie recommendation system. A well-designed recommendation algorithm can significantly enhance user experience by suggesting movies that align with their preferences, leading to higher platform retention and usage. Recommendation systems face the challenge of balancing exploration (discovering new movies) with exploitation (recommending known favourites) to maximize user satisfaction over time.


# Scenario

Imagine a leading online movie streaming platform, TrendMovie Inc., that aims to become the go-to destination for personalized movie recommendations. The platform features a vast collection of movies catering to diverse audiences. TrendMovie Inc. wants to optimize its recommendation strategy to deliver maximum user satisfaction while maintaining a high level of engagement. Each movie recommendation is treated as an interaction with the user, and their feedback is used to refine the recommendation strategy dynamically.


# Objective

Your objective is to design and implement a recommendation system using Multi-Armed Bandit (MAB) algorithms to maximize cumulative user satisfaction. The system should dynamically allocate recommendations by learning user preferences in real-time, striking the right balance between exploration and exploitation.


# Dataset

The dataset contains user ratings for a variety of movies. Key columns in the dataset include:
*   **User ID:** A unique identifier for each user.
*   **Movie ID:** A unique identifier for each.
*   **Rating:** A score provided by the user for a movie (on a scale of 1 to 5).
*   **Timestamp:** The time when the rating was given (optional for this assignment).

***Link for accessing dataset:***
https://drive.google.com/file/d/1gfobhqlVCw8Oo52JCiYpEBGhG5k7cWBr/view?usp=drive_link


# Environment Details

**Arms:** Each movie represents an "arm" in the MAB framework. The probability of a movie being liked by a user is initially unknown and will be estimated based on user feedback during the interactions.
For example:

Arm 1: Movie A

Arm 2: Movie B

Arm 3: Movie C

... and so on, for all movies in the dataset.

**Reward Function:**
The reward function is defined based on user ratings:

***Reward = 1:*** The user rates the movie high star (e.g., 4 or 5 stars).

***Reward = 0:*** The user rates the movie low star (e.g., 1, 2, or 3 stars).


**Assumptions:**

Run simulations for 1000 iterations for each policy


# Requirements and Deliverables:
Implement the Multi-Arm Bandit Problem for the given above scenario for all the below mentioned policy methods.

In [11]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
import math
import plotly.graph_objects as go
import matplotlib.pyplot as plt

### Initialize constants

In [12]:
# Initialize value function and policy
n_iterations = 1000
epsilon_lst = [0.1, 0.2, 0.5]

data_path = "TrendMovie.csv" 

# Load Dataset (0.5M)

In [13]:
# Code for Dataset loading and print dataset statistics
try:
    df = pd.read_csv(data_path)
    print(f"Dataset successfully loaded with {df.shape[0]} rows and {df.shape[1]} columns.")
except FileNotFoundError:
    raise FileNotFoundError(f"The file at {data_path} was not found. Please check the path.")

# Display dataset statistics
print("Dataset Statistics:")
print(df.describe())

df['reward'] = (df['rating'] >= 4).astype(int)
print("\nReward Column Added:")
print(df[['movieId', 'rating', 'reward']].head())

Dataset successfully loaded with 100836 rows and 4 columns.
Dataset Statistics:
              userId        movieId         rating     timestamp
count  100836.000000  100836.000000  100836.000000  1.008360e+05
mean      326.127564   19435.295718       3.501557  1.205946e+09
std       182.618491   35530.987199       1.042529  2.162610e+08
min         1.000000       1.000000       0.500000  8.281246e+08
25%       177.000000    1199.000000       3.000000  1.019124e+09
50%       325.000000    2991.000000       3.500000  1.186087e+09
75%       477.000000    8122.000000       4.000000  1.435994e+09
max       610.000000  193609.000000       5.000000  1.537799e+09

Reward Column Added:
   movieId  rating  reward
0        1     4.0       1
1        3     4.0       1
2        6     4.0       1
3       47     5.0       1
4       50     5.0       1


# Design a Movie Environment (0.5M)

In [14]:
# Code for Dataset loading and print dataset statistics along with reward function

class MovieEnvironment:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.movies = self.df['movieId'].unique()
        self.movie_stats = self.df.groupby('movieId').agg({
            'reward': ['count', 'mean']
        }).reset_index()
        self.movie_stats.columns = ['MovieID', 'total_ratings', 'mean_reward']
        self.movie_to_arm = {movie: idx for idx, movie in enumerate(self.movies)}
        self.arm_to_movie = {idx: movie for movie, idx in self.movie_to_arm.items()}
        self.n_arms = len(self.movies)

    def get_reward(self, movie_id: int) -> int:
        movie_stats = self.movie_stats[self.movie_stats['MovieID'] == movie_id].iloc[0]
        return np.random.binomial(1, movie_stats['mean_reward'])

# Using Random Policy (0.5M)
Implement a random policy for movie recommendations and print each iteration. (Mandatory)

In [15]:
#  run the environment with an agent that is guided by a random policy
# Implement Random Policy
def random_policy(env: MovieEnvironment, n_iterations: int):
    rewards = []
    print("\nRandom Policy:")
    for t in range(n_iterations):
        chosen_arm = np.random.randint(env.n_arms)
        movie_id = env.arm_to_movie[chosen_arm]
        reward = env.get_reward(movie_id)
        rewards.append(reward)
        print(f"Iteration {t + 1}: Recommended Movie {movie_id}, Reward: {reward}")
    return rewards


# Using Greedy Policy (1M)
Implement a greedy policy that always recommends the movie with the highest estimated reward and print each iteration. (Mandatory)

In [16]:
#  run the environment with an agent that is guided by a greedy policy
# Implement Greedy Policy
def greedy_policy(env: MovieEnvironment, n_iterations: int):
    counts = np.zeros(env.n_arms)
    rewards = np.zeros(env.n_arms)
    all_rewards = []
    print("\nGreedy Policy:")
    for t in range(n_iterations):
        estimates = rewards / (counts + 1e-9)  # Avoid division by zero
        chosen_arm = np.argmax(estimates)
        movie_id = env.arm_to_movie[chosen_arm]
        reward = env.get_reward(movie_id)
        counts[chosen_arm] += 1
        rewards[chosen_arm] += reward
        all_rewards.append(reward)
        print(f"Iteration {t + 1}: Recommended Movie {movie_id}, Reward: {reward}")
    return all_rewards


# Using Epsilon-Greedy Policy (1.5M)
Implement the epsilon-greedy policy, where with probability ε you explore (recommend a random movie) and with probability (1-ε) you exploit (recommend the best-known movie). Try with ε =0.1, 0.2, 0.5 and print each iteration. What value of ε yields the best performance? (Mandatory)

In [17]:
#  run the environment with an agent that is guided by a epsilon-greedy policy
# Implement Epsilon-Greedy Policy
def epsilon_greedy_policy(env: MovieEnvironment, n_iterations: int, epsilon: float):
    counts = np.zeros(env.n_arms)
    rewards = np.zeros(env.n_arms)
    all_rewards = []
    print(f"\nEpsilon-Greedy Policy (ε={epsilon}):")
    for t in range(n_iterations):
        if np.random.random() < epsilon:
            chosen_arm = np.random.randint(env.n_arms)  # Explore
        else:
            estimates = rewards / (counts + 1e-9)  # Exploit
            chosen_arm = np.argmax(estimates)
        movie_id = env.arm_to_movie[chosen_arm]
        reward = env.get_reward(movie_id)
        counts[chosen_arm] += 1
        rewards[chosen_arm] += reward
        all_rewards.append(reward)
        print(f"Iteration {t + 1}: Recommended Movie {movie_id}, Reward: {reward}")
    return all_rewards


# Using UCB (1M)
Implement the UCB algorithm for movie recommendations and print each iteration. (Mandatory)

In [18]:
#  run the environment with an agent that is guided by a UCB
# Implement UCB Policy
def ucb_policy(env: MovieEnvironment, n_iterations: int):
    counts = np.zeros(env.n_arms)
    rewards = np.zeros(env.n_arms)
    all_rewards = []
    print("\nUCB Policy:")
    for t in range(1, n_iterations + 1):
        ucb_values = rewards / (counts + 1e-9) + np.sqrt(2 * np.log(t) / (counts + 1e-9))
        chosen_arm = np.argmax(ucb_values)
        movie_id = env.arm_to_movie[chosen_arm]
        reward = env.get_reward(movie_id)
        counts[chosen_arm] += 1
        rewards[chosen_arm] += reward
        all_rewards.append(reward)
        print(f"Iteration {t}: Recommended Movie {movie_id}, Reward: {reward}")
    return all_rewards


# Plot the cumulative rewards for all policies on a single graph to compare their performance. (0.5M)

In [19]:
# Plot Cumulative Rewards
# Plot Cumulative Rewards using Plotly
def plot_cumulative_rewards_plotly(results: Dict[str, List[int]], n_iterations: int):
    fig = go.Figure()

    for policy, rewards in results.items():
        cumulative_rewards = np.cumsum(rewards)
        fig.add_trace(go.Scatter(
            x=list(range(1, n_iterations + 1)),
            y=cumulative_rewards,
            mode='lines',
            name=policy
        ))

    fig.update_layout(
        title="Cumulative Rewards Comparison Across Policies",
        xaxis_title="Iterations",
        yaxis_title="Cumulative Reward",
        legend_title="Policy",
        template="plotly_white"
    )
    fig.show()


In [20]:
# Main Execution
def main():
    env = MovieEnvironment(df)
    results = {}

    # Run all policies
    results['Random'] = random_policy(env, n_iterations)
    results['Greedy'] = greedy_policy(env, n_iterations)
    for epsilon in epsilon_lst:
        results[f"Epsilon-Greedy (ε={epsilon})"] = epsilon_greedy_policy(env, n_iterations, epsilon)
    results['UCB'] = ucb_policy(env, n_iterations)

    # Plot results
    plot_cumulative_rewards_plotly(results, n_iterations)


if __name__ == "__main__":
    main()



Random Policy:
Iteration 1: Recommended Movie 2316, Reward: 0
Iteration 2: Recommended Movie 88699, Reward: 0
Iteration 3: Recommended Movie 43560, Reward: 0
Iteration 4: Recommended Movie 98160, Reward: 0
Iteration 5: Recommended Movie 159779, Reward: 1
Iteration 6: Recommended Movie 58554, Reward: 1
Iteration 7: Recommended Movie 169670, Reward: 0
Iteration 8: Recommended Movie 3323, Reward: 1
Iteration 9: Recommended Movie 2389, Reward: 0
Iteration 10: Recommended Movie 127134, Reward: 0
Iteration 11: Recommended Movie 5076, Reward: 0
Iteration 12: Recommended Movie 60950, Reward: 0


Iteration 13: Recommended Movie 62374, Reward: 1
Iteration 14: Recommended Movie 3255, Reward: 1
Iteration 15: Recommended Movie 4047, Reward: 0
Iteration 16: Recommended Movie 109191, Reward: 0
Iteration 17: Recommended Movie 5833, Reward: 1
Iteration 18: Recommended Movie 99846, Reward: 0
Iteration 19: Recommended Movie 9005, Reward: 0
Iteration 20: Recommended Movie 32554, Reward: 0
Iteration 21: Recommended Movie 65135, Reward: 0
Iteration 22: Recommended Movie 5423, Reward: 1
Iteration 23: Recommended Movie 26870, Reward: 0
Iteration 24: Recommended Movie 62344, Reward: 0
Iteration 25: Recommended Movie 79695, Reward: 0
Iteration 26: Recommended Movie 188, Reward: 1
Iteration 27: Recommended Movie 96084, Reward: 1
Iteration 28: Recommended Movie 45728, Reward: 0
Iteration 29: Recommended Movie 87869, Reward: 0
Iteration 30: Recommended Movie 8405, Reward: 1
Iteration 31: Recommended Movie 128089, Reward: 1
Iteration 32: Recommended Movie 71129, Reward: 0
Iteration 33: Recommended 

# Conclusion (0.5M)

Determine which policy performs the best based on cumulative reward. Provide a concise conclusion (250 words) summarizing the decision-making process and the trade-offs between exploration and exploitation.

`----write below this line------`

The ε-Greedy (ε=0.1) policy performs best with a cumulative reward of 933.0, indicating it strikes the optimal balance between exploration and exploitation. A smaller ε value encourages more exploitation, maximizing rewards from known good options while still allowing limited exploration.

The ε-Greedy (ε=0.2) policy shows a slight decline in reward (860.0) due to increased exploration, which dilutes the focus on higher-reward arms. The ε-Greedy (ε=0.5) policy performs even worse (704.0) as the focus on exploration leads to suboptimal choices and reduced cumulative rewards.

The Greedy policy (687.0) only exploits known rewards but misses out on potential better options due to its lack of exploration. UCB (458.0) performs better than random but underperforms compared to ε-Greedy, as it tries to balance exploration and exploitation in a less effective way.

Finally, the Random policy (376.0) performs the worst, as it lacks any strategy and randomly chooses arms without regard to their potential rewards.

In conclusion, the ε-Greedy (ε=0.1) policy offers the best performance, providing the right mix of exploration and exploitation, while other policies either focus too much on one or fail to strike the optimal balance.

The trade-off between exploration and exploitation is about balancing the discovery of new actions (exploration) and choosing known actions that yield high rewards (exploitation).

Exploration helps discover potentially better options but can reduce short-term rewards.
Exploitation focuses on known rewards, maximizing immediate gain but may miss better options.
    